# **Ray Tracing** in **GL (or other) Shading Language**

## Group members
* Musaab Al-Bakry
* MingMing Sun
* Rebecca Jacobs
* Andrew Pickner (?)


![](https://raw.githubusercontent.com/larsbrinkhoff/glsl-sphere-tracing/gif/cube.gif)

Below is a snippet of code from the Sphere Tracing Algorithm (a version of Ray Tracing):

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def intersect_sphere(O, D, S, R):
    # Return the distance from O to the intersection of the ray (O, D) with the 
    # sphere (S, R), or +inf if there is no intersection.
    # O and S are 3D points, D (direction) is a normalized vector, R is a scalar.
    a = np.dot(D, D)
    OS = O - S
    b = 2 * np.dot(D, OS)
    c = np.dot(OS, OS) - R * R
    disc = b * b - 4 * a * c
    if disc > 0:
        distSqrt = np.sqrt(disc)
        q = (-b - distSqrt) / 2.0 if b < 0 else (-b + distSqrt) / 2.0
        t0 = q / a
        t1 = c / q
        t0, t1 = min(t0, t1), max(t0, t1)
        if t1 >= 0:
            return t1 if t0 < 0 else t0
    return np.inf

def add_sphere(position, radius, color):
    return dict(type='sphere', position=np.array(position), 
        radius=np.array(radius), color=np.array(color), reflection=.5)
    
def add_plane(position, normal):
    return dict(type='plane', position=np.array(position), 
        normal=np.array(normal),
        color=lambda M: (color_plane0 
            if (int(M[0] * 2) % 2) == (int(M[2] * 2) % 2) else color_plane1),
        diffuse_c=.75, specular_c=.5, reflection=.25)
    



and include code snippets

```c
double square(double x) {
    return x*x;
}
```
or code cells

In [5]:
def square(x):
    return x*x

print(f'square(3) = {square(3)}')

square(3) = 9


## About Sphere Tracing

Sphere tracing uses functions that return the distance to their implicit surfaces, called distance surfaces **(1.2)**, to get a sequence of points that converge if and only if the ray intersects the implicit surface. The image is then created using unbounding spheres. An unbounding volume (sphere) surrounds the space that doesn't contain the object. Therefore, the name "sphere tracing" is used for this algorithm.

  ![](https://i.stack.imgur.com/dFHFY.png)

Sphere tracing doesn't require the ability to evaluate the derivative of the function, unlike other algorithms that use rootfinding techniques such as Newton's method. It only requires a bound on the magnitude of the derivative. However, the function itself must be Lipschitz **(1.3)** and continuous. This makes it superior to methods that use polygonization because it can detect disconnected sections of the surface.

Alternative methods for implicit surface rendering do exist. However, some of the other methods use polygonization **(1.1)** which is problematic because it can be costly and inefficient for representation of a relatively simple surface. Polygonizations also have the potential to overlook disconnected or detailed sections of the surface. 

Although it can be less desirabe on simple objects made of quadrics and polygons, sphere tracing proves to be extremely useful and robust when rendering more complicated geometry.  It can also be used on a wider variety of surfaces.

**1.1 Polygonization** - approximating an implicit surface using discrete sample points and structuring those points using methods such as triangulation to create a satisfactory model.

**1.2 Distance Surfaces**- functions that measure or bound the geometric distance to their implicit surfaces 

**1.3 Lipschitz Function**

A function $f$ : $\mathbb{R}^3$ $\Rightarrow$ $\mathbb{R}$ is $Lipschitz$ over a domain $D$ if and only if for all $\mathbf{x}$, $\mathbf{y}$  $\in$  $D$,
there exists a positive finite constant $\lambda$ such that

|$f(x) - f(y)$|  $\leq$  $\lambda$ ||$\mathbf{x}$ - $\mathbf{y}$ ||.
                            
The $Lipschitz$ $constant$ denoted Lip $f$, is the minimum $\lambda$ satisfying the above equation.



## About GLSL (OpenGL Shading Language)

GLSL has syntax based on C

    for(int i=0;i<max_iter;i++)
    {
        vec4 mz = vec4(z.x,-z.y,-z.z,-z.w);
        // derivative
        dz[0]=vec4(dot(mz,dz[0]),z.x*dz[0].yzw+dz[0].x*z.yzw);
        dz[1]=vec4(dot(mz,dz[1]),z.x*dz[1].yzw+dz[1].x*z.yzw);
        dz[2]=vec4(dot(mz,dz[2]),z.x*dz[2].yzw+dz[2].x*z.yzw);
        //dz[3]=vec4(dot(mz,dz[3]),z.x*dz[3].yzw+dz[3].x*z.yzw);

        // z = z2 + c
        nz.x=dot(z, mz);
        nz.yzw=2.0*z.x*z.yzw;
        z=nz+c;

        if(dot(z,z)>4.0)
            break;
        }

Repository link: 
https://github.com/larsbrinkhoff/glsl-sphere-tracing

GLSL was created by the OpenGL Architecture Review Board for developers in order to have better control over 3-D rendering without having to use languages that are hardware specific. It is used mostly for developing video games in this context, but can be used for other purposes like computer aided design.

GLSL has language bindings to C, C++, Java, and other languages and therefore has great adaptability.

## Sphere tracing as it appears in GLSL

GLSL is the main language used to execute the sphere tracing algorithm. It can be used to test performance and run different variations of the algorithm and is helpful in conducting various research.

Convergence for the sphere tracing algorithm is strictly linear and is mostly dependent on surface edges. Sometimes the distance to the surface is extremely small compared to the distance to the ray intersection, which slows convergence and causes the algorithm to be quite costly. 

## Sphere tracing in other Shading Languages 







## Methods 


## Results and interpretation

## Conclusions and open questions